In [3]:
#psycopg2

In [1]:
import pandas as pd
import psycopg2 as pg

In [2]:
# Credetials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
pwd = 'cdspa'

In [3]:
def open_conn(host, port, database, username, pwd):
    conn = pg.connect(user=username,
               password=pwd,
               host=host,
               port=port,
               database=database)
    return conn

In [11]:
cursor.execute("ROLLBACK")
conn.commit()

# 1.0 Schema Query

In [4]:
cursor = conn.cursor()

query_schema = """
    select nspname
    from pg_catalog.pg_namespace
"""

cursor.execute(query_schema)

NameError: name 'conn' is not defined

In [14]:
record = cursor.fetchall()
cursor.close()
conn.close()

In [15]:
record

[('pg_toast',),
 ('pg_temp_1',),
 ('pg_toast_temp_1',),
 ('pg_catalog',),
 ('information_schema',),
 ('public',),
 ('pa004',),
 ('pg_temp_4',),
 ('pg_toast_temp_4',),
 ('pa005',),
 ('pg_temp_9',),
 ('pg_toast_temp_9',)]

# 2.0 Table Query

In [23]:
conn = open_conn(host=host, port=port, database=database, username=username, pwd=pwd)

cursor = conn.cursor()

query_tables = """
    select tablename
    from pg_tables
    where schemaname = 'pa004'
"""

cursor.execute(query_tables)

record = cursor.fetchall()
print(record)
cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0 Query Data using Psycopg2

In [24]:
conn = open_conn(host=host, port=port, database=database, username=username, pwd=pwd)

cursor = conn.cursor()

query_tables = """
    select *
    from pa004.users u
    where u.age < 30
    limit 10
"""

cursor.execute(query_tables)

record = cursor.fetchall()
print(record)
cursor.close()
conn.close()

[(4, 'Male', 21, 11.0, 152.0), (5, 'Female', 29, 41.0, 152.0), (6, 'Female', 24, 33.0, 160.0), (7, 'Male', 23, 11.0, 152.0), (9, 'Female', 24, 3.0, 152.0), (12, 'Female', 24, 50.0, 152.0), (17, 'Female', 25, 45.0, 160.0), (18, 'Female', 25, 35.0, 152.0), (23, 'Male', 23, 50.0, 152.0), (26, 'Female', 21, 28.0, 152.0)]


In [28]:
data = pd.DataFrame(record)
data

,0,1,2,3,4
0,4,Male,21,11.0,152.0
1,5,Female,29,41.0,152.0
2,6,Female,24,33.0,160.0
3,7,Male,23,11.0,152.0
4,9,Female,24,3.0,152.0
5,12,Female,24,50.0,152.0
6,17,Female,25,45.0,160.0
7,18,Female,25,35.0,152.0
8,23,Male,23,50.0,152.0
9,26,Female,21,28.0,152.0


# 4.0 Query Data using Pandas

In [14]:
conn = open_conn(host=host, port=port, database=database, username=username, pwd=pwd)

query_tables_users = """
    select *
    from pa004.users u inner join pa004.vehicle v on (u.id = v.id)
                       inner join pa004.insurance i on (u.id = i.id)
"""

df = pd.read_sql(query_tables_users, conn)

In [15]:
df.columns

Index(['id', 'gender', 'age', 'region_code', 'policy_sales_channel', 'id',
       'driving_license', 'vehicle_age', 'vehicle_damage', 'id',
       'previously_insured', 'annual_premium', 'vintage', 'response'],
      dtype='object')

In [16]:
df1 = df.drop(df.columns[[5,9]], axis=1)
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [8]:
df1.columns

Index(['gender', 'age', 'region_code', 'policy_sales_channel',
       'driving_license', 'vehicle_age', 'vehicle_damage',
       'previously_insured', 'annual_premium', 'vintage', 'response'],
      dtype='object')

In [19]:
df1.to_csv("data/train.csv", encoding='utf-8', index=False)